Installing necessary Libraries

In [ ]:
!pip install ultralytics
!pip install opencv-python-headless
!pip install lap
!pip install filterpy
!pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/AI_Assignment/

15sec_input_720p.mp4  best.pt  broadcast.mp4  tacticam.mp4


Setup YOLO

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load the YOLO model
model = YOLO('/content/drive/MyDrive/AI_Assignment/best.pt')

# Load the video
video_path = '/content/drive/MyDrive/AI_Assignment/15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

# Output video writer
output_path = '/content/drive/MyDrive/AI_Assignment/output.mp4'
os.makedirs('/content', exist_ok=True)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (1280, 720))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])

        if cls == 0:  # Assuming class 0 is 'player'
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Player {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("Detection video saved at:", output_path)


0: 384x640 1 ball, 16 players, 2 referees, 3032.0ms
Speed: 3.9ms preprocess, 3032.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 3017.4ms
Speed: 3.5ms preprocess, 3017.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 3044.7ms
Speed: 3.7ms preprocess, 3044.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 4796.2ms
Speed: 4.0ms preprocess, 4796.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3022.3ms
Speed: 4.3ms preprocess, 3022.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 3015.9ms
Speed: 4.2ms preprocess, 3015.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 3028.9ms
Speed: 3.4ms preprocess, 3028.9ms inference, 1.2ms 

Integrate Deep SORT to Assign Player IDs

In [ ]:
!pip install deep_sort_realtime

In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
import cv2
import os

# Load YOLO model
model = YOLO('/content/drive/MyDrive/AI_Assignment/best.pt')

# Load video
video_path = '/content/drive/MyDrive/AI_Assignment/15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

# Output
output_path = '/content/drive/MyDrive/AI_Assignment/output_with_ids.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (1280, 720))

# Initialize Deep SORT tracker
tracker = DeepSort(max_age=30)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO Detection
    results = model(frame)[0]
    detections = []

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])

        if cls == 0:  # Assuming class 0 is 'player'
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'player'))

    # Deep SORT tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    out.write(frame)

cap.release()
out.release()
print("Tracking video with player IDs saved at:", output_path)



0: 384x640 1 ball, 16 players, 2 referees, 3104.8ms
Speed: 4.8ms preprocess, 3104.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 3407.5ms
Speed: 3.7ms preprocess, 3407.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 4378.1ms
Speed: 3.6ms preprocess, 4378.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3018.5ms
Speed: 3.2ms preprocess, 3018.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3016.8ms
Speed: 3.3ms preprocess, 3016.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 3051.1ms
Speed: 3.3ms preprocess, 3051.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 4710.2ms
Speed: 4.0ms preprocess, 4710.2ms inference, 1.3ms 